# Setup

Import basic packages and bring in lookup tables for clinical conditions and medications and pre-defined mapping functions.

#### Packages

In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
# set number of rows pandas dataframe displays
pd.set_option('display.max_rows', 3000)

#### Lookup tables

In [2]:
lkp_med = pd.read_csv('S:\CALIBER_17_205R\MSc\Oliver\Lookup Tables\lookup_medical_table.csv', sep=',')
lkp_med_code_dic = pickle.load(open('S:\CALIBER_17_205R\MSc\Oliver\Python Code\lkp_med_code_dic.p', 'rb'))
lkp_med_term_dic = pickle.load(open('S:\CALIBER_17_205R\MSc\Oliver\Python Code\lkp_med_term_dic.p', 'rb'))

lkp_bnf = pd.read_csv('S:\CALIBER_17_205R\MSc\Oliver\Lookup Tables\lookup_bnf_table.csv', 
                      sep=',', dtype={'bnfcode':str, 'bnf':str})

lkp_prd = pd.read_csv('S:\CALIBER_17_205R\MSc\Oliver\Lookup Tables\lookup_product_table.csv', sep=',', 
                      low_memory=False)

#### Mappings from helper_functions

In [3]:
from helper_functions import medcode_to_read, medcode_to_term, read_to_medcode, cprd_to_bnf, bnf_to_chapter

#### Patid lists relating to readcodes of interest

Bring in required data and get high-level positive/negative patid lists

In [4]:
df_c_max = pd.read_csv(r'S:\CALIBER_17_205R\MSc\Oliver\MySQL Output\5_analysis\clinical_5_max_84_after.txt', 
                       sep='\t', names=['patid','eventdate','medcode','episode','enttype','adid'])
df_c_max['readcode'] = df_c_max['medcode'].apply(medcode_to_read, args=(lkp_med_code_dic,))
df_c_max['readterm'] = df_c_max['medcode'].apply(medcode_to_term, args=(lkp_med_term_dic,))
df_c_max = df_c_max.drop(columns = ['medcode'])

df_f = pd.read_csv(r'S:\CALIBER_17_205R\MSc\Oliver\MySQL Output\5_analysis\flags_5.txt', sep='\t')
df_f = df_f[['patid','min_eventdate_5','max_eventdate_5_less_6_month','max_eventdate_5','max_eventdate',
             'stopped_in_data']]

patids_1 = df_f[df_f['stopped_in_data']==1]['patid'].unique()
patids_0 = df_f[df_f['stopped_in_data']==0]['patid'].unique()

Patid lists relating to generic readcodes

In [5]:
# medication review terms taken from Medication Review - Best Practice Guidelinees (Dudley NHS), with additional codes
patids_med_rvw = df_c_max[df_c_max['readterm'].isin(['Medication review done by pharmacist',
                                                     'Medication review',
                                                     'Medication review done',
                                                     'Medication review with patient',
                                                     'Medication review with nurse',
                                                     'Medication review by pharmacy technician',
                                                     'Medication review of medical notes',
                                                     'Medication review without patient'])]['patid'].unique()
print(f'patids_med_rvw: {len(patids_med_rvw)}')

patids_med_cng = df_c_max[df_c_max['readterm'].isin(['Medication changed'])]['patid'].unique()
print(f'patids_med_cng: {len(patids_med_cng)}')

patids_cnd_imp = df_c_max[df_c_max['readterm'].isin(["Patient's condition improved"])]['patid'].unique()
print(f'patids_cnd_imp: {len(patids_cnd_imp)}')

patids_eha = df_c_max[df_c_max['readterm'].isin(['Emergency hospital admission'])]['patid'].unique()
print(f'patids_eha: {len(patids_eha)}')

patids_cas = df_c_max[df_c_max['readterm'].isin(['Seen in hospital casualty'])]['patid'].unique()
print(f'patids_cas: {len(patids_cas)}')

patids_ane = df_c_max[df_c_max['readterm'].isin(['Seen by accident and emergency doctor'])]['patid'].unique()
print(f'patids_ane: {len(patids_ane)}')

# read terms for death taken from LSHTM clinical code lists
read_death = pd.read_csv(r'S:\CALIBER_17_205R\MSc\Oliver\Clinical Code Lists\Clinical_codelist_Read_death.txt',
                         sep='\t')
patids_died = df_c_max[df_c_max['readcode'].isin(list(read_death['readcode']))]['patid'].unique()
print(f'patids_died: {len(patids_died)}')

patids_med_rvw: 83295
patids_med_cng: 6745
patids_cnd_imp: 7332
patids_eha: 11524
patids_cas: 31753
patids_ane: 4119
patids_died: 72154


Patid lists relating to medication specific readcodes

In [6]:
patids_cou = df_c_max[df_c_max['readterm'].isin(['Cough',
                                                 'C/O - cough',
                                                 'Dry cough'])]['patid'].unique()
print(f'patids_cou: {len(patids_cou)}')

patids_cou: 51736


### Define labels/targets of interest

Create a selection of labels/targets to fit classifiers to and to test predictive ability against.

#### Positive instances

Focussed on combinations of patients being taken off the medication coinciding with:
- an admission to hospital, further narrowed down by additional filters for medication reviews, medication changes coded in to the data and indications that the patient has not been taken off the medication because their condition has improved
- a complaint of a known side-effect of the drug, further narrowed down with additional filters as above

In [7]:
# 1a. patients taken off medication AND preceeded by an emergency hospital admission
patids_v1a_pos = list(set.intersection(set(patids_1), set(patids_eha)))

# 1b. patients taken off medication AND preceeded by hospital casualty
patids_v1b_pos = list(set.intersection(set(patids_1), set(patids_cas)))

# 1c. patients taken off medication AND preceeded by being seen by an A&E doctor
patids_v1c_pos = list(set.intersection(set(patids_1), set(patids_ane)))

# 1d. patients taken off medication AND preceeded by an emergency admission OR hospital casualty OR seen by A&E doc
patids_v1d_pos = list(set.union(set.union(set(patids_v1a_pos), set(patids_v1b_pos)), set(patids_v1c_pos)))

# 1e. patients from 1d. AND medication review
patids_v1e_pos = list(set.intersection(set(patids_v1d_pos), set(patids_med_rvw)))

# 1f. patients from 1d. AND medication change
patids_v1f_pos = list(set.intersection(set(patids_v1d_pos), set(patids_med_cng)))

# 1g. patients from 1d. AND NOT condition improved
patids_cnd_not_imp = list(set(patids_1) - set(patids_cnd_imp))
patids_v1g_pos = list(set.intersection(set(patids_v1d_pos), set(patids_cnd_not_imp)))


# 2a. patients taken off medication AND preceeded by a cough
patids_v2a_pos = list(set.intersection(set(patids_1), set(patids_cou)))

# 2b. patients taken off medication AND preceeded by a cough AND medication review
patids_v2b_pos = list(set.intersection(set(patids_v2a_pos), set(patids_med_rvw)))

# 2c. patients taken off medication AND preceeded by a cough AND medication change
patids_v2c_pos = list(set.intersection(set(patids_v2a_pos), set(patids_med_cng)))

# 2d. patients taken off medication AND preceeded by a cough AND NOT condition improved
patids_cnd_not_imp = list(set(patids_1) - set(patids_cnd_imp))
patids_v2d_pos = list(set.intersection(set(patids_v2a_pos), set(patids_cnd_not_imp)))

In [8]:
patids_pos = list(set.union(set(patids_v1d_pos), set(patids_v2a_pos)))

#### Negative instances

Make baseline set as general as possible whilst remaining distinct to the positive instances of the target labels. 

A suitable baseline group has been chosen for each positive group e.g. a patient developing a side effect after the last prescription of medication in the data but without an emergency hospital admission would be a suitable baseline case for v1 but not for v2.

Due to restrictions on the computational capacity avaliable, unblanaced data is dealt with by matching the size of the baseline/negative isntances class to the size of the positive class.

In [9]:
# patients NOT taken off medication, with NOT hospital admission following the last prescription AND NOT dead
n_v1 = len(patids_v1d_pos)
np.random.seed(0)
patids_v1_neg = np.random.choice(list(set(patids_0)-set(patids_eha)-set(patids_cas)-set(patids_ane)-set(patids_died)), 
                                 size=n_v1, replace=False)

# patients NOT taken off medication, with NOT a cough following the last prescription AND NOT dead
n_v2 = len(patids_v2a_pos)
np.random.seed(0)
patids_v2_neg = np.random.choice(list(set(patids_0)-set(patids_cou)-set(patids_died)), 
                                 size=n_v2, replace=False)

In [10]:
patids_neg = list(set.union(set(patids_v1_neg), set(patids_v2_neg)))

#### Full patient list

In [11]:
patids = list(set.union(set(patids_pos), set(patids_neg)))

# Bring in pre-processed data to build stat_dic from

Data has been pre-processed in SQL prior to bringing in to this workbook.

In [12]:
# clinical data from start up to 84 days after the last date of prescription of medication
df_c = pd.read_csv(r'S:\CALIBER_17_205R\MSc\Oliver\MySQL Output\5_analysis\clinical_5.txt', 
                   sep='\t', names=['patid','eventdate','medcode','episode','enttype','adid'])

# therapy data from minium date of presctiption to 84 days prior
df_t_min = pd.read_csv(r'S:\CALIBER_17_205R\MSc\Oliver\MySQL Output\5_analysis\therapy_5_min_84_before.txt', 
                       sep='\t', names=['patid','eventdate','consid','prodcode','dosageid','bnfcode','qty','numdays'],
                       dtype={'qty':str})

# therapy data from 6 months prior to last date of prescription of medication to 84 days prior
df_t_max_6 = pd.read_csv(r'S:\CALIBER_17_205R\MSc\Oliver\MySQL Output\5_analysis\therapy_5_max_6_mnth_84_before.txt', 
                       sep='\t', names=['patid','eventdate','consid','prodcode','dosageid','bnfcode','qty','numdays'],
                       dtype={'qty':str})

# patient data for all patients recieving at least one instance of the medication
df_p = pd.read_csv(r'S:\CALIBER_17_205R\MSc\Oliver\MySQL Output\5_analysis\patient_5.txt', sep='\t')

Due to the size of the data and the limits on computational capacity we immediately cut the data down to the size required for subsequent analysis as per the full patient listing above.

In [13]:
df_c = df_c[df_c['patid'].isin(patids)]
df_c['readcode'] = df_c['medcode'].apply(medcode_to_read, args=(lkp_med_code_dic,))
df_c['readterm'] = df_c['medcode'].apply(medcode_to_term, args=(lkp_med_term_dic,))
df_c = df_c.drop(columns = ['medcode'])

df_t_min = df_t_min[df_t_min['patid'].isin(patids)]

df_t_max_6 = df_t_max_6[df_t_max_6['patid'].isin(patids)]

df_f = df_f[df_f['patid'].isin(patids)]

df_p = df_p[df_p['patid'].isin(patids)]

# Build stat_dic

#### Mappings from helper_functions

In [14]:
from helper_functions import build_stat_dic

Construct target_dic from the labels/targets specified up top, and feed to the build_stat_dic function along with the medication specific pre-processed data.

In [15]:
target_dic = {'patids_v1_neg' :patids_v1_neg,
              'patids_v1a_pos':patids_v1a_pos,
              'patids_v1b_pos':patids_v1b_pos,
              'patids_v1c_pos':patids_v1c_pos,
              'patids_v1d_pos':patids_v1d_pos,
              'patids_v1e_pos':patids_v1e_pos,
              'patids_v1f_pos':patids_v1f_pos,
              'patids_v1g_pos':patids_v1g_pos,
              'patids_v2_neg' :patids_v2_neg,
              'patids_v2a_pos':patids_v2a_pos,
              'patids_v2b_pos':patids_v2b_pos,
              'patids_v2c_pos':patids_v2c_pos,
              'patids_v2d_pos':patids_v2d_pos}

stat_dic_5 = build_stat_dic(patids, str(5), df_c, df_t_min, df_t_max_6, df_f, df_p, target_dic, tdqm_display=True)

100%|████████████████████████████████████████████████████████████████████████| 125969/125969 [5:06:05<00:00,  6.86it/s]


#### Save stat_dic

In [16]:
pickle.dump(stat_dic_5, open('stat_dic_5.p', 'wb'))